# Imports

In [1]:
# %% Imports y configuración
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
from pathlib import Path
import json

# Agregar el directorio raíz al path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..'))

# Imports del proyecto
from model_ddp.utils.sistem_fun import (
    load_config,
    get_data_path,
    get_artifact_path,
    get_report_path,
    create_experiment_id,
    ensure_directories,
    save_experiment_metadata
)

from model_ddp.simulations.gaussian_simulator import (
    RegressionSimulator,
    SimulationConfig,
    RBFKernel,
    MaternKernel,
    PeriodicKernel,
    TransformationFunctions
)

# Modelos
from model_ddp.models.PSBP_normal_v3 import PSBPNormal

# Metricas y graficas 
from model_ddp.fit.metrics import regression_metrics
from model_ddp.graphics.plots_regression import plot_regression_analysis
from model_ddp.graphics.plots_traces import plot_hyperparameter_traces
from model_ddp.graphics.plots_aplication import plot_credible_intervals

config=load_config()

Implementación PSBP en C++ Exitosa


Los experimentos realizados:
- 1 Feature Linear (n=200) -> 1200 (200 burn)
- 2 Feature Linear (n=200) -> 1200 (200 burn)
- 2 Feature Cuadraticos (n=400) -> 1200 (200 burn)
- 3 Feature Linear, variar Kernel Mater (n=800) ->2400 (400 burn)

Busco evaluar el tiempo de desempeño de la clase LSBP para el caso normal y las mejoras que realice en C++, aqui podra ver como aplicar la mayor parte de las funciones.

In [2]:
# 1 Feature Linear (n=200) -> 1200 (200 burn)
# 2 Feature Linear (n=200) -> 1200 (200 burn)
# 2 Feature Cuadraticos (n=400) -> 1200 (200 burn)
# 3 Feature Linear, variar Kernel Mater (n=800) ->2400 (400 burn)

# Experimento I 

In [3]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_psbp_001"
SIM_REAL = "simulation"

In [4]:
# Parámetros de ejecución de experimentos 
CARACTERISTICAS = "1 Feature Linear (n=200) -> 1200 (200 burn)"
EXPERIMENT_ID = create_experiment_id("psbp_001_exp_01")

In [5]:
##################################################
# Registrar Experimento
##################################################

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""Experimento: {CARACTERISTICAS}"""
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")


✓ Experimento registrado en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\versioning\experiment_registry.md


In [6]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{EXPERIMENT_ID}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=200,
    n_features=1,
    x_range=(0.0, 100.0),
    noise_std=0.2,
    random_state=234
)

# Definir kernel (RBF)
kernel = RBFKernel(
    length_scale=5.0,
    variance=1.0
)


transformation = TransformationFunctions.linear(
    coefficients= [2],
    intercept=[10]
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (200, 1)
  Media por feature: [0.35866669]
  Std por feature: [0.82864792]

Estadísticas de Y:
  Shape: (200,)
  Media: 10.7327
  Std: 1.6713
  Min: 6.4454
  Max: 13.9424
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\psbp_001_exp_01_20251230_015430/_data.csv


### Modelo

In [7]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO PSBPNormal...")
print("="*60)

# Crear instancia del modelo
psbp_model = PSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=15,
    verbose=True,
    normalize=False
)

# Ejecutar MCMC
trace = psbp_model.run(
    iterations=1200,          # Iteraciones totales
    burnin=200               # Burn-in
)

print("\n" + "="*60)
print("PSBP COMPLETADO")
print("="*60)


EJECUTANDO PSBPNormal...
Using C++ acceleration
Iter 100/1200: K_eff=4, H=102, μ=0.33, μ₀=11.08, κ₀=0.12, a₀=20.00, b₀=5.93
Iter 200/1200: K_eff=3, H=102, μ=0.47, μ₀=8.55, κ₀=0.17, a₀=20.00, b₀=6.92
Iter 300/1200: K_eff=3, H=102, μ=0.64, μ₀=9.36, κ₀=0.48, a₀=20.00, b₀=10.14
Iter 400/1200: K_eff=3, H=102, μ=0.57, μ₀=9.25, κ₀=0.02, a₀=20.00, b₀=8.66
Iter 500/1200: K_eff=5, H=102, μ=0.58, μ₀=5.98, κ₀=0.04, a₀=20.00, b₀=7.19
Iter 600/1200: K_eff=3, H=102, μ=0.64, μ₀=8.49, κ₀=0.14, a₀=19.35, b₀=10.20
Iter 700/1200: K_eff=3, H=102, μ=0.49, μ₀=9.97, κ₀=0.13, a₀=20.00, b₀=9.15
Iter 800/1200: K_eff=3, H=102, μ=0.64, μ₀=11.81, κ₀=0.07, a₀=20.00, b₀=8.68
Iter 900/1200: K_eff=3, H=102, μ=0.42, μ₀=7.74, κ₀=0.14, a₀=20.00, b₀=8.23
Iter 1000/1200: K_eff=4, H=102, μ=0.71, μ₀=13.51, κ₀=0.01, a₀=20.00, b₀=6.25
Iter 1100/1200: K_eff=4, H=102, μ=0.71, μ₀=9.95, κ₀=0.08, a₀=20.00, b₀=3.75
Iter 1200/1200: K_eff=4, H=102, μ=0.88, μ₀=12.11, κ₀=0.08, a₀=20.00, b₀=5.73

PSBP COMPLETADO


### Guardado Modelo

In [8]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto PSBPNormal)
model_file = carpeta_modelo / "psbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(psbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = psbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir todos los valores a tipos serializables
    summary_json = {}
    for k, v in summary.items():
        if isinstance(v, tuple):
            # Convertir elementos de la tupla
            v0 = v[0].tolist() if hasattr(v[0], 'tolist') else v[0]
            v1 = v[1].tolist() if hasattr(v[1], 'tolist') else v[1]
            summary_json[k] = {'mean': v0, 'std': v1}
        elif hasattr(v, 'tolist'):
            summary_json[k] = v.tolist()
        else:
            summary_json[k] = v
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'PSBPNormal',
    'data_shape': {
        'n': psbp_model.n,
        'p': psbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': psbp_model.n_grid
    },
    'priors': {
        'mu_prior': [float(psbp_model.mu_mu), float(psbp_model.tau_mu_inv)],
        'mu0_prior': [float(psbp_model.m0), float(psbp_model.s02)],
        'kappa0_prior': [float(psbp_model.alpha_kappa), float(psbp_model.beta_kappa)],
        'a0_prior': [float(psbp_model.alpha_a), float(psbp_model.beta_a)],
        'b0_prior': [float(psbp_model.alpha_b), float(psbp_model.beta_b)],
        'psi_prior_mean': psbp_model.mu_psi.tolist() if hasattr(psbp_model.mu_psi, 'tolist') else list(psbp_model.mu_psi),
        'psi_prior_prec': psbp_model.tau_psi.tolist() if hasattr(psbp_model.tau_psi, 'tolist') else list(psbp_model.tau_psi)
    },
    'final_stats': {
        'H_final': psbp_model.H,
        'n_clusters_mean': float(summary['n_clusters'][0]) if isinstance(summary['n_clusters'][0], (np.number, float, int)) else summary['n_clusters'][0],
        'n_clusters_std': float(summary['n_clusters'][1]) if isinstance(summary['n_clusters'][1], (np.number, float, int)) else summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': float(np.mean(psbp_model.mh_acceptance['alpha'][-100:])) if psbp_model.mh_acceptance['alpha'] else 0.0,
        'psi': float(np.mean(psbp_model.mh_acceptance['psi'][-100:])) if psbp_model.mh_acceptance['psi'] else 0.0,
        'kappa0': float(np.mean(psbp_model.mh_acceptance['kappa0'][-100:])) if psbp_model.mh_acceptance['kappa0'] else 0.0,
        'a0': float(np.mean(psbp_model.mh_acceptance['a0'][-100:])) if psbp_model.mh_acceptance['a0'] else 0.0
    },
    'other_params': {
        'psi_positive': psbp_model.psi_positive,
        'estimate_psi_hyperpriors': psbp_model.estimate_psi_hyperpriors,
        'use_cpp': psbp_model.use_cpp,
        'verbose': psbp_model.verbose
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': float(psbp_model.y_mean) if hasattr(psbp_model.y_mean, '__float__') else psbp_model.y_mean,
    'y_std': float(psbp_model.y_std) if hasattr(psbp_model.y_std, '__float__') else psbp_model.y_std,
    'X_mean': psbp_model.X_mean.tolist() if hasattr(psbp_model.X_mean, 'tolist') else list(psbp_model.X_mean),
    'X_std': psbp_model.X_std.tolist() if hasattr(psbp_model.X_std, 'tolist') else list(psbp_model.X_std)
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)


GUARDANDO MODELO Y RESULTADOS...
✓ Modelo guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251230_015430\psbp_model.pkl
✓ Trazas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251230_015430\trace.pkl
✓ Resumen posterior guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251230_015430\posterior_summary.json
✓ Metadatos guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251230_015430\metadata.json
✓ Datos de normalización guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251230_015430\normalization.pkl

MODELO GUARDADO EN: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251230_015430


### Predicciones, grafica de Predicciones y guardado

In [9]:
##################################################
#  FIT  (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO PREDICCIONES...")
print("="*60)

# Hacer predicciones con el modelo entrenado
y_pred_mean, y_pred_std = psbp_model.predict_mean(
    X_new=datos.drop(columns=["Y"]).values,
    n_samples=1000
)

y_true = datos["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})
predictions_file = carpeta_reportes / "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"✓ Predicciones guardadas: {predictions_file}")

##################################################
# Gráficas de Fit (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS...")
print("="*60)
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Training Set")
]

plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="PSBP Normal"
)

print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


GENERANDO PREDICCIONES...



📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :   0.431244
  RMSE    :   0.656692
  MAE     :   0.477787
  R2      :   0.845614
  MAPE    :   4.489795
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_01_20251230_015430\metrics.json
✓ Predicciones guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_01_20251230_015430\predictions.csv

GENERANDO GRÁFICAS...
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_01_20251230_015430


### Otros analisis

In [10]:
##################################################
# Trazas 
##################################################
# Parametros a ver las trazas
hyperparams = [
    ('mu', 'μ (Intercepto stick-breaking)'),
    ('mu0', 'μ₀ (Media base)'),
    ('kappa0', 'κ₀ (Precisión relativa)'),
    ('a0', 'a₀ (Shape σ²)'),
    ('b0', 'b₀ (Scale σ²)'),    
    ('n_clusters', 'Número de Clusters')  
]

plot_hyperparameter_traces(
    trace=trace,
    param_config=hyperparams,
    output_path=carpeta_graficas / f"{EXPERIMENT_ID}.png",
    title="Trazas PSBP Normal"
)

##################################################
# Gráfica de intervalo predicción media 
##################################################

# Versión simple
plot_credible_intervals(
    y_true=y_true,
    y_pred_mean=y_pred_mean,
    y_pred_std=y_pred_std,
    output_path=carpeta_graficas / "intervalos_credibilidad.png",
    sort_by='y_true'
)

print(f"✓ Todas las gráficas guardadas en: {carpeta_graficas}")

c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:78: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:83: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.savefig(output_path, dpi=300, bbox_inches='tight')


✓ Todas las gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_01_20251230_015430


# Experimento II 

In [11]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_psbp_001"
SIM_REAL = "simulation"

In [12]:
# Parámetros de ejecución de experimentos 
CARACTERISTICAS = "2 Feature Linear (n=200) -> 1200 (200 burn)"
EXPERIMENT_ID = create_experiment_id("psbp_001_exp_02")

In [13]:
##################################################
# Registrar Experimento
##################################################

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""Experimento: {CARACTERISTICAS}"""
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")


✓ Experimento registrado en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\versioning\experiment_registry.md


In [14]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{EXPERIMENT_ID}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=200,
    n_features=2,
    x_range=(0.0, 100.0),
    noise_std=0.2,
    random_state=234
)

# Definir kernel (RBF)
kernel = RBFKernel(
    length_scale=5.0,
    variance=1.0
)


transformation = TransformationFunctions.linear(
    coefficients= [2,15],
    intercept=[10]
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (200, 2)
  Media por feature: [ 0.35866669 -0.30311795]
  Std por feature: [0.82864792 0.75455656]

Estadísticas de Y:
  Shape: (200,)
  Media: 6.1859
  Std: 11.7161
  Min: -17.2658
  Max: 33.8075
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\psbp_001_exp_02_20251230_015648/_data.csv


## Modelo 

In [15]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
psbp_model = PSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=15,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = psbp_model.run(
    iterations=1200,          # Iteraciones totales
    burnin=200               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPNormal...
Using C++ acceleration
Iter 100/1200: K_eff=5, H=102, μ=-0.55, μ₀=-6.19, κ₀=0.06, a₀=0.59, b₀=4.19
Iter 200/1200: K_eff=6, H=102, μ=-0.62, μ₀=11.21, κ₀=0.08, a₀=0.50, b₀=2.81
Iter 300/1200: K_eff=6, H=102, μ=-0.66, μ₀=8.98, κ₀=0.06, a₀=0.52, b₀=3.21
Iter 400/1200: K_eff=5, H=102, μ=-0.33, μ₀=9.12, κ₀=0.09, a₀=0.55, b₀=2.29
Iter 500/1200: K_eff=5, H=102, μ=-0.49, μ₀=5.71, κ₀=0.08, a₀=0.50, b₀=3.59
Iter 600/1200: K_eff=6, H=102, μ=-0.48, μ₀=14.82, κ₀=0.08, a₀=0.50, b₀=2.22
Iter 700/1200: K_eff=5, H=102, μ=-0.44, μ₀=6.27, κ₀=0.07, a₀=0.57, b₀=1.54
Iter 800/1200: K_eff=5, H=102, μ=-0.51, μ₀=-2.26, κ₀=0.15, a₀=0.55, b₀=5.66
Iter 900/1200: K_eff=5, H=102, μ=-0.73, μ₀=3.22, κ₀=0.01, a₀=0.62, b₀=3.57
Iter 1000/1200: K_eff=5, H=102, μ=-0.26, μ₀=2.90, κ₀=0.08, a₀=0.50, b₀=3.52
Iter 1100/1200: K_eff=6, H=102, μ=-0.15, μ₀=15.12, κ₀=0.16, a₀=0.54, b₀=2.71
Iter 1200/1200: K_eff=5, H=102, μ=-0.08, μ₀=11.11, κ₀=0.17, a₀=0.65, b₀=4.10

LSBP COMPLETADO


## Guardado Modelo

In [16]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto PSBPNormal)
model_file = carpeta_modelo / "psbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(psbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = psbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir todos los valores a tipos serializables
    summary_json = {}
    for k, v in summary.items():
        if isinstance(v, tuple):
            # Convertir elementos de la tupla
            v0 = v[0].tolist() if hasattr(v[0], 'tolist') else v[0]
            v1 = v[1].tolist() if hasattr(v[1], 'tolist') else v[1]
            summary_json[k] = {'mean': v0, 'std': v1}
        elif hasattr(v, 'tolist'):
            summary_json[k] = v.tolist()
        else:
            summary_json[k] = v
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'PSBPNormal',
    'data_shape': {
        'n': psbp_model.n,
        'p': psbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': psbp_model.n_grid
    },
    'priors': {
        'mu_prior': [float(psbp_model.mu_mu), float(psbp_model.tau_mu_inv)],
        'mu0_prior': [float(psbp_model.m0), float(psbp_model.s02)],
        'kappa0_prior': [float(psbp_model.alpha_kappa), float(psbp_model.beta_kappa)],
        'a0_prior': [float(psbp_model.alpha_a), float(psbp_model.beta_a)],
        'b0_prior': [float(psbp_model.alpha_b), float(psbp_model.beta_b)],
        'psi_prior_mean': psbp_model.mu_psi.tolist() if hasattr(psbp_model.mu_psi, 'tolist') else list(psbp_model.mu_psi),
        'psi_prior_prec': psbp_model.tau_psi.tolist() if hasattr(psbp_model.tau_psi, 'tolist') else list(psbp_model.tau_psi)
    },
    'final_stats': {
        'H_final': psbp_model.H,
        'n_clusters_mean': float(summary['n_clusters'][0]) if isinstance(summary['n_clusters'][0], (np.number, float, int)) else summary['n_clusters'][0],
        'n_clusters_std': float(summary['n_clusters'][1]) if isinstance(summary['n_clusters'][1], (np.number, float, int)) else summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': float(np.mean(psbp_model.mh_acceptance['alpha'][-100:])) if psbp_model.mh_acceptance['alpha'] else 0.0,
        'psi': float(np.mean(psbp_model.mh_acceptance['psi'][-100:])) if psbp_model.mh_acceptance['psi'] else 0.0,
        'kappa0': float(np.mean(psbp_model.mh_acceptance['kappa0'][-100:])) if psbp_model.mh_acceptance['kappa0'] else 0.0,
        'a0': float(np.mean(psbp_model.mh_acceptance['a0'][-100:])) if psbp_model.mh_acceptance['a0'] else 0.0
    },
    'other_params': {
        'psi_positive': psbp_model.psi_positive,
        'estimate_psi_hyperpriors': psbp_model.estimate_psi_hyperpriors,
        'use_cpp': psbp_model.use_cpp,
        'verbose': psbp_model.verbose
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': float(psbp_model.y_mean) if hasattr(psbp_model.y_mean, '__float__') else psbp_model.y_mean,
    'y_std': float(psbp_model.y_std) if hasattr(psbp_model.y_std, '__float__') else psbp_model.y_std,
    'X_mean': psbp_model.X_mean.tolist() if hasattr(psbp_model.X_mean, 'tolist') else list(psbp_model.X_mean),
    'X_std': psbp_model.X_std.tolist() if hasattr(psbp_model.X_std, 'tolist') else list(psbp_model.X_std)
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)


GUARDANDO MODELO Y RESULTADOS...
✓ Modelo guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251230_015648\psbp_model.pkl
✓ Trazas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251230_015648\trace.pkl
✓ Resumen posterior guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251230_015648\posterior_summary.json
✓ Metadatos guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251230_015648\metadata.json
✓ Datos de normalización guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251230_015648\normalization.pkl

MODELO GUARDADO EN: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251230_015648


## Pred, graf

In [17]:
##################################################
#  FIT  (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO PREDICCIONES...")
print("="*60)

# Hacer predicciones con el modelo entrenado
y_pred_mean, y_pred_std = psbp_model.predict_mean(
    X_new=datos.drop(columns=["Y"]).values,
    n_samples=1000
)

y_true = datos["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})
predictions_file = carpeta_reportes / "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"✓ Predicciones guardadas: {predictions_file}")

##################################################
# Gráficas de Fit (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS...")
print("="*60)
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Training Set")
]

plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="PSBP Normal"
)

print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


GENERANDO PREDICCIONES...

📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :   8.783606
  RMSE    :   2.963715
  MAE     :   2.465739
  R2      :   0.936010
  MAPE    :  63.599896
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_02_20251230_015648\metrics.json
✓ Predicciones guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_02_20251230_015648\predictions.csv

GENERANDO GRÁFICAS...
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_02_20251230_015648


## Otros analisis

In [18]:
##################################################
# Trazas 
##################################################
# Parametros a ver las trazas
hyperparams = [
    ('mu', 'μ (Intercepto stick-breaking)'),
    ('mu0', 'μ₀ (Media base)'),
    ('kappa0', 'κ₀ (Precisión relativa)'),
    ('a0', 'a₀ (Shape σ²)'),
    ('b0', 'b₀ (Scale σ²)'),    
    ('n_clusters', 'Número de Clusters')   
]

plot_hyperparameter_traces(
    trace=trace,
    param_config=hyperparams,
    output_path=carpeta_graficas / f"{EXPERIMENT_ID}.png",
    title="Trazas PSBP Normal"
)

##################################################
# Gráfica de intervalo predicción media 
##################################################

# Versión simple
plot_credible_intervals(
    y_true=y_true,
    y_pred_mean=y_pred_mean,
    y_pred_std=y_pred_std,
    output_path=carpeta_graficas / "intervalos_credibilidad.png",
    sort_by='y_true'
)

print(f"✓ Todas las gráficas guardadas en: {carpeta_graficas}")

c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:78: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:83: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.savefig(output_path, dpi=300, bbox_inches='tight')


✓ Todas las gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_02_20251230_015648


# Experimento III

In [19]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_psbp_001"
SIM_REAL = "simulation"

In [20]:
# Parámetros de ejecución de experimentos 
CARACTERISTICAS = "2 Feature Cuadraticos (n=400) -> 1200 (200 burn)"
EXPERIMENT_ID = create_experiment_id("psbp_001_exp_03")

In [21]:
##################################################
# Registrar Experimento
##################################################

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""Experimento: {CARACTERISTICAS}"""
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")


✓ Experimento registrado en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\versioning\experiment_registry.md


In [22]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{EXPERIMENT_ID}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=400,
    n_features=2,
    x_range=(0.0, 100.0),
    noise_std=0.2,
    random_state=234
)

# Definir kernel (RBF)
kernel = RBFKernel(
    length_scale=5.0,
    variance=1.0
)


transformation = TransformationFunctions.polynomial(
    degree=2
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (400, 2)
  Media por feature: [ 0.14187066 -0.50272967]
  Std por feature: [0.78558034 0.87377361]

Estadísticas de Y:
  Shape: (400,)
  Media: 1.6552
  Std: 1.9636
  Min: -0.4204
  Max: 9.8443
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\psbp_001_exp_03_20251230_015941/_data.csv


## Modelo

In [23]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
psbp_model = PSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=15,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = psbp_model.run(
    iterations=1200,          # Iteraciones totales
    burnin=200               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPNormal...
Using C++ acceleration
Iter 100/1200: K_eff=2, H=102, μ=-0.08, μ₀=1.39, κ₀=0.22, a₀=1.45, b₀=0.78
Iter 200/1200: K_eff=2, H=102, μ=-0.18, μ₀=1.87, κ₀=0.78, a₀=1.26, b₀=0.87
Iter 300/1200: K_eff=2, H=102, μ=-0.19, μ₀=1.96, κ₀=0.21, a₀=1.09, b₀=0.78
Iter 400/1200: K_eff=2, H=102, μ=-0.03, μ₀=0.65, κ₀=0.47, a₀=0.84, b₀=0.69
Iter 500/1200: K_eff=2, H=102, μ=-0.05, μ₀=1.57, κ₀=0.74, a₀=1.51, b₀=1.53
Iter 600/1200: K_eff=2, H=102, μ=0.07, μ₀=0.50, κ₀=0.55, a₀=0.93, b₀=0.57
Iter 700/1200: K_eff=2, H=102, μ=0.12, μ₀=1.44, κ₀=0.41, a₀=0.85, b₀=0.42
Iter 800/1200: K_eff=2, H=102, μ=0.30, μ₀=0.42, κ₀=0.76, a₀=0.62, b₀=0.47
Iter 900/1200: K_eff=2, H=102, μ=0.58, μ₀=0.93, κ₀=0.91, a₀=2.09, b₀=1.87
Iter 1000/1200: K_eff=2, H=102, μ=0.77, μ₀=-0.87, κ₀=0.06, a₀=0.71, b₀=0.82
Iter 1100/1200: K_eff=2, H=102, μ=0.52, μ₀=-0.06, κ₀=0.69, a₀=1.49, b₀=1.34
Iter 1200/1200: K_eff=2, H=102, μ=0.39, μ₀=-0.22, κ₀=0.19, a₀=0.50, b₀=0.22

LSBP COMPLETADO


## Guardado Modelo

In [24]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto PSBPNormal)
model_file = carpeta_modelo / "psbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(psbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = psbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir todos los valores a tipos serializables
    summary_json = {}
    for k, v in summary.items():
        if isinstance(v, tuple):
            # Convertir elementos de la tupla
            v0 = v[0].tolist() if hasattr(v[0], 'tolist') else v[0]
            v1 = v[1].tolist() if hasattr(v[1], 'tolist') else v[1]
            summary_json[k] = {'mean': v0, 'std': v1}
        elif hasattr(v, 'tolist'):
            summary_json[k] = v.tolist()
        else:
            summary_json[k] = v
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'PSBPNormal',
    'data_shape': {
        'n': psbp_model.n,
        'p': psbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': psbp_model.n_grid
    },
    'priors': {
        'mu_prior': [float(psbp_model.mu_mu), float(psbp_model.tau_mu_inv)],
        'mu0_prior': [float(psbp_model.m0), float(psbp_model.s02)],
        'kappa0_prior': [float(psbp_model.alpha_kappa), float(psbp_model.beta_kappa)],
        'a0_prior': [float(psbp_model.alpha_a), float(psbp_model.beta_a)],
        'b0_prior': [float(psbp_model.alpha_b), float(psbp_model.beta_b)],
        'psi_prior_mean': psbp_model.mu_psi.tolist() if hasattr(psbp_model.mu_psi, 'tolist') else list(psbp_model.mu_psi),
        'psi_prior_prec': psbp_model.tau_psi.tolist() if hasattr(psbp_model.tau_psi, 'tolist') else list(psbp_model.tau_psi)
    },
    'final_stats': {
        'H_final': psbp_model.H,
        'n_clusters_mean': float(summary['n_clusters'][0]) if isinstance(summary['n_clusters'][0], (np.number, float, int)) else summary['n_clusters'][0],
        'n_clusters_std': float(summary['n_clusters'][1]) if isinstance(summary['n_clusters'][1], (np.number, float, int)) else summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': float(np.mean(psbp_model.mh_acceptance['alpha'][-100:])) if psbp_model.mh_acceptance['alpha'] else 0.0,
        'psi': float(np.mean(psbp_model.mh_acceptance['psi'][-100:])) if psbp_model.mh_acceptance['psi'] else 0.0,
        'kappa0': float(np.mean(psbp_model.mh_acceptance['kappa0'][-100:])) if psbp_model.mh_acceptance['kappa0'] else 0.0,
        'a0': float(np.mean(psbp_model.mh_acceptance['a0'][-100:])) if psbp_model.mh_acceptance['a0'] else 0.0
    },
    'other_params': {
        'psi_positive': psbp_model.psi_positive,
        'estimate_psi_hyperpriors': psbp_model.estimate_psi_hyperpriors,
        'use_cpp': psbp_model.use_cpp,
        'verbose': psbp_model.verbose
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': float(psbp_model.y_mean) if hasattr(psbp_model.y_mean, '__float__') else psbp_model.y_mean,
    'y_std': float(psbp_model.y_std) if hasattr(psbp_model.y_std, '__float__') else psbp_model.y_std,
    'X_mean': psbp_model.X_mean.tolist() if hasattr(psbp_model.X_mean, 'tolist') else list(psbp_model.X_mean),
    'X_std': psbp_model.X_std.tolist() if hasattr(psbp_model.X_std, 'tolist') else list(psbp_model.X_std)
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)


GUARDANDO MODELO Y RESULTADOS...
✓ Modelo guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251230_015941\psbp_model.pkl
✓ Trazas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251230_015941\trace.pkl
✓ Resumen posterior guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251230_015941\posterior_summary.json
✓ Metadatos guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251230_015941\metadata.json
✓ Datos de normalización guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251230_015941\normalization.pkl

MODELO GUARDADO EN: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251230_015941


## pred Graf

In [25]:
##################################################
#  FIT  (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO PREDICCIONES...")
print("="*60)

# Hacer predicciones con el modelo entrenado
y_pred_mean, y_pred_std = psbp_model.predict_mean(
    X_new=datos.drop(columns=["Y"]).values,
    n_samples=1000
)

y_true = datos["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})
predictions_file = carpeta_reportes / "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"✓ Predicciones guardadas: {predictions_file}")

##################################################
# Gráficas de Fit (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS...")
print("="*60)
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Training Set")
]

plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="PSBP Normal"
)

print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


GENERANDO PREDICCIONES...

📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :   0.890531
  RMSE    :   0.943679
  MAE     :   0.561154
  R2      :   0.769048
  MAPE    : 160.503276
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_03_20251230_015941\metrics.json
✓ Predicciones guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_03_20251230_015941\predictions.csv

GENERANDO GRÁFICAS...
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_03_20251230_015941


## Otros 

In [26]:
##################################################
# Trazas 
##################################################
# Parametros a ver las trazas
hyperparams = [
    ('mu', 'μ (Intercepto stick-breaking)'),
    ('mu0', 'μ₀ (Media base)'),
    ('kappa0', 'κ₀ (Precisión relativa)'),
    ('a0', 'a₀ (Shape σ²)'),
    ('b0', 'b₀ (Scale σ²)'),    
    ('n_clusters', 'Número de Clusters')   
]

plot_hyperparameter_traces(
    trace=trace,
    param_config=hyperparams,
    output_path=carpeta_graficas / f"{EXPERIMENT_ID}.png",
    title="Trazas PSBP Normal"
)

##################################################
# Gráfica de intervalo predicción media 
##################################################

# Versión simple
plot_credible_intervals(
    y_true=y_true,
    y_pred_mean=y_pred_mean,
    y_pred_std=y_pred_std,
    output_path=carpeta_graficas / "intervalos_credibilidad.png",
    sort_by='y_true'
)

print(f"✓ Todas las gráficas guardadas en: {carpeta_graficas}")

c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:78: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:83: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.savefig(output_path, dpi=300, bbox_inches='tight')


✓ Todas las gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_03_20251230_015941


# Experimento IV

In [27]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_psbp_001"
SIM_REAL = "simulation"

In [28]:
# Parámetros de ejecución de experimentos 
CARACTERISTICAS = "3 Feature Linear, variar Kernel Mater (n=800) ->2400 (400 burn)"
EXPERIMENT_ID = create_experiment_id("psbp_001_exp_04")

In [29]:
##################################################
# Registrar Experimento
##################################################

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""Experimento: {CARACTERISTICAS}"""
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")


✓ Experimento registrado en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\versioning\experiment_registry.md


In [30]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{EXPERIMENT_ID}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=800,
    n_features=3,
    x_range=(0.0, 100.0),
    noise_std=0.2,
    random_state=234
)

# Definir kernel (RBF)
kernel = MaternKernel(
    length_scale=5.0,
    variance=2.0
)


transformation = TransformationFunctions.linear(
    coefficients=[15,-5,3],
    intercept=[100]
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (800, 3)
  Media por feature: [-0.23289032  0.35779777  0.13681772]
  Std por feature: [1.11561901 0.93767766 1.07967819]

Estadísticas de Y:
  Shape: (800,)
  Media: 95.1227
  Std: 16.0322
  Min: 65.3149
  Max: 136.1139
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\psbp_001_exp_04_20251230_020359/_data.csv


## Modelo

In [ ]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
psbp_model = PSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=30,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = psbp_model.run(
    iterations=2400,          # Iteraciones totales
    burnin=400               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPNormal...
Using C++ acceleration


## Guardado Modelo

In [ ]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto PSBPNormal)
model_file = carpeta_modelo / "psbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(psbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = psbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir todos los valores a tipos serializables
    summary_json = {}
    for k, v in summary.items():
        if isinstance(v, tuple):
            # Convertir elementos de la tupla
            v0 = v[0].tolist() if hasattr(v[0], 'tolist') else v[0]
            v1 = v[1].tolist() if hasattr(v[1], 'tolist') else v[1]
            summary_json[k] = {'mean': v0, 'std': v1}
        elif hasattr(v, 'tolist'):
            summary_json[k] = v.tolist()
        else:
            summary_json[k] = v
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'PSBPNormal',
    'data_shape': {
        'n': psbp_model.n,
        'p': psbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': psbp_model.n_grid
    },
    'priors': {
        'mu_prior': [float(psbp_model.mu_mu), float(psbp_model.tau_mu_inv)],
        'mu0_prior': [float(psbp_model.m0), float(psbp_model.s02)],
        'kappa0_prior': [float(psbp_model.alpha_kappa), float(psbp_model.beta_kappa)],
        'a0_prior': [float(psbp_model.alpha_a), float(psbp_model.beta_a)],
        'b0_prior': [float(psbp_model.alpha_b), float(psbp_model.beta_b)],
        'psi_prior_mean': psbp_model.mu_psi.tolist() if hasattr(psbp_model.mu_psi, 'tolist') else list(psbp_model.mu_psi),
        'psi_prior_prec': psbp_model.tau_psi.tolist() if hasattr(psbp_model.tau_psi, 'tolist') else list(psbp_model.tau_psi)
    },
    'final_stats': {
        'H_final': psbp_model.H,
        'n_clusters_mean': float(summary['n_clusters'][0]) if isinstance(summary['n_clusters'][0], (np.number, float, int)) else summary['n_clusters'][0],
        'n_clusters_std': float(summary['n_clusters'][1]) if isinstance(summary['n_clusters'][1], (np.number, float, int)) else summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': float(np.mean(psbp_model.mh_acceptance['alpha'][-100:])) if psbp_model.mh_acceptance['alpha'] else 0.0,
        'psi': float(np.mean(psbp_model.mh_acceptance['psi'][-100:])) if psbp_model.mh_acceptance['psi'] else 0.0,
        'kappa0': float(np.mean(psbp_model.mh_acceptance['kappa0'][-100:])) if psbp_model.mh_acceptance['kappa0'] else 0.0,
        'a0': float(np.mean(psbp_model.mh_acceptance['a0'][-100:])) if psbp_model.mh_acceptance['a0'] else 0.0
    },
    'other_params': {
        'psi_positive': psbp_model.psi_positive,
        'estimate_psi_hyperpriors': psbp_model.estimate_psi_hyperpriors,
        'use_cpp': psbp_model.use_cpp,
        'verbose': psbp_model.verbose
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': float(psbp_model.y_mean) if hasattr(psbp_model.y_mean, '__float__') else psbp_model.y_mean,
    'y_std': float(psbp_model.y_std) if hasattr(psbp_model.y_std, '__float__') else psbp_model.y_std,
    'X_mean': psbp_model.X_mean.tolist() if hasattr(psbp_model.X_mean, 'tolist') else list(psbp_model.X_mean),
    'X_std': psbp_model.X_std.tolist() if hasattr(psbp_model.X_std, 'tolist') else list(psbp_model.X_std)
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)


GUARDANDO MODELO Y RESULTADOS...
✓ Modelo guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\psbp_model.pkl
✓ Trazas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\trace.pkl
✓ Resumen posterior guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\posterior_summary.json
✓ Metadatos guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\metadata.json
✓ Datos de normalización guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\normalization.pkl

MODELO GUARDADO EN: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318


## Pred, graf

In [ ]:
##################################################
#  FIT  (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO PREDICCIONES...")
print("="*60)

# Hacer predicciones con el modelo entrenado
y_pred_mean, y_pred_std = psbp_model.predict_mean(
    X_new=datos.drop(columns=["Y"]).values,
    n_samples=1000
)

y_true = datos["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})
predictions_file = carpeta_reportes / "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"✓ Predicciones guardadas: {predictions_file}")

##################################################
# Gráficas de Fit (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS...")
print("="*60)
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Training Set")
]

plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="PSBP Normal"
)

print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


GENERANDO PREDICCIONES...

📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :  43.983346
  RMSE    :   6.631994
  MAE     :   4.890547
  R2      :   0.828879
  MAPE    :   5.492428
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_04_20251226_210318\metrics.json
✓ Predicciones guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_04_20251226_210318\predictions.csv

GENERANDO GRÁFICAS...
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_04_20251226_210318


## Otros

In [ ]:
##################################################
# Trazas 
##################################################
# Parametros a ver las trazas
hyperparams = [
    ('mu', 'μ (Intercepto stick-breaking)'),
    ('mu0', 'μ₀ (Media base)'),
    ('kappa0', 'κ₀ (Precisión relativa)'),
    ('a0', 'a₀ (Shape σ²)'),
    ('b0', 'b₀ (Scale σ²)'),    
    ('n_clusters', 'Número de Clusters')   
]

plot_hyperparameter_traces(
    trace=trace,
    param_config=hyperparams,
    output_path=carpeta_graficas / f"{EXPERIMENT_ID}.png",
    title="Trazas PSBP Normal"
)

##################################################
# Gráfica de intervalo predicción media 
##################################################

# Versión simple
plot_credible_intervals(
    y_true=y_true,
    y_pred_mean=y_pred_mean,
    y_pred_std=y_pred_std,
    output_path=carpeta_graficas / "intervalos_credibilidad.png",
    sort_by='y_true'
)

print(f"✓ Todas las gráficas guardadas en: {carpeta_graficas}")

c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:78: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:83: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.savefig(output_path, dpi=300, bbox_inches='tight')


✓ Todas las gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_04_20251226_210318
